In [1]:
import os
import json
import openai
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [12]:
"""
Outline of medical analysis example using ChatGPT API
- create a database of example patients
- Ask GPT to get example patient from database 
- Request Progress notes, Discharge Summary, Consultation Report, 
    Laboratory Report, Radiology Reports, Surgery Reports, 
    Immunization records, allergy records, medication records, and vital sign records for the patient
- Summarize and Analyze reports and return a high level summary of the patient and diagnostic predictions

Use Completions API, Chat Completions API, and function Calling capabilities
"""

'\nOutline of medical analysis example using ChatGPT API\n- create a database of example patients\n- Ask GPT to get example patient from database \n- Request Progress notes, Discharge Summary, Consultation Report, \n    Laboratory Report, Radiology Reports, Surgery Reports, \n    Immunization records, allergy records, medication records, and vital sign records for the patient\n- Summarize and Analyze reports and return a high level summary of the patient and diagnostic predictions\n\nUse Completions API, Chat Completions API, and function Calling capabilities\n'

In [13]:
patient_dict_messages = [
    {"role": "system", "content": "You are a system that creates information about medical patients with real conditions"},
    {"role": "user", "content": "Please provide a list of 5 patients with real and different medical conditions. " + 
            "Format your response in JSON format with the following keys: Name, Age, Gender, Occupation, Medical Condition, Medical History, Symptoms, Diagnostic Journey, Treatment, Impact on Daily Life"}
]

In [14]:
patient_dict_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages = patient_dict_messages,
    temperature=0.7
)
patient_dict = patient_dict_response["choices"][0]["message"]["content"]
print(patient_dict)
patient_dict = json.loads(patient_dict)

{
  "patients": [
    {
      "Name": "John Smith",
      "Age": 45,
      "Gender": "Male",
      "Occupation": "Accountant",
      "Medical Condition": "Type 2 Diabetes",
      "Medical History": "Family history of diabetes, sedentary lifestyle, poor diet",
      "Symptoms": "Frequent urination, increased thirst, fatigue, blurred vision",
      "Diagnostic Journey": "Routine blood tests revealed high blood sugar levels, followed by further tests to confirm diagnosis",
      "Treatment": "Lifestyle modifications (diet, exercise), oral medication (metformin)",
      "Impact on Daily Life": "Requires regular monitoring of blood sugar levels, dietary restrictions, medication adherence"
    },
    {
      "Name": "Emily Johnson",
      "Age": 30,
      "Gender": "Female",
      "Occupation": "Teacher",
      "Medical Condition": "Asthma",
      "Medical History": "Childhood history of allergies, exposure to environmental triggers",
      "Symptoms": "Shortness of breath, wheezing, coughin

In [15]:
def get_patient(user_id):
    """This is meant to simulate retrieving patient records from a database"""
    if user_id in range(0,4):
        return json.dumps(patient_dict["patients"][user_id])
    raise Exception("User not found")
    

In [16]:
messages = [{'role': 'system', 'content': 'You are an intelligent medical assistant'}, 
            {'role': 'user', 'content': 'Please a patient from our internal database with a user_id between 0 and 4.'}]
functions = [{'name': 'get_patient', 
              'description': 'Get patient details from internal database.',
              'parameters': {
                  'type': 'object',
                  'properties': {
                      'user_id':  {
                          'type': 'integer',
                          'description': 'The user_id of the patient'
                        }
                    },
                'required': ['user_id']
                }
            }
        ]

In [17]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
response_message = response["choices"][0]["message"]


In [18]:
if response_message.get("function_call"):
    available_fncts = {
        "get_patient": get_patient,
    }
    fctn_name = response_message.get("function_call").get("name")
    fnctn_call = available_fncts.get(fctn_name)
    function_args = json.loads(response_message.get("function_call").get("arguments"))
    function_response = json.loads(fnctn_call(user_id=function_args.get("user_id")))
    diagnostic_journey = function_response.get("Diagnostic Journey")
    medical_history = function_response.get("Medical History")

In [20]:
print(medical_history)
print(diagnostic_journey)

medical_reports_messages = [
    {"role": "system", "content": "You are a system that creates medical reports"},
    {"role": "user", "content": "Given the following medical history please create progress notes, medication history, immunization records, allergy records, and vital sign records." + 
                        " If the records do not exist, create them." + 
                        "additionally, extract any medical examinations from the diagnosic journey and provide reports for them." + 
                        f"\n Medical History: \n[{medical_history}]" +
                        f"\n Diagnostic Journey: \n[{diagnostic_journey}]"},
]

Age-related degeneration, previous joint injuries
Physical examination, X-rays, blood tests to rule out other conditions


In [21]:
medical_reports_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=medical_reports_messages
)
medical_reports_content = medical_reports_response.choices[0]["message"]["content"]
print(medical_reports_content)

**Medical Progress Notes:**

Patient Name: [Patient Name]
Gender: [Patient Gender]
Date of Birth: [Patient Date of Birth]
Medical History: Age-related degeneration, previous joint injuries

**Progress Note 1:**
Date: [Date]
Subjective: The patient presents with complaints of joint pain and limited mobility, especially in the affected joints.
Objective: Physical examination reveals mild swelling and tenderness in the affected joints. Limited range of motion is also observed. X-rays and blood tests have been ordered to rule out other conditions.
Assessment: Suspected age-related degeneration and previous joint injuries.
Plan: X-rays and blood tests are scheduled. Prescribed pain medication and recommended physical therapy for pain management and improving mobility.

**Progress Note 2:**
Date: [Date]
Subjective: The patient reports moderate improvement in joint pain and mobility after completing two weeks of physical therapy.
Objective: Physical examination shows reduced swelling and tend

In [22]:
summary_analysis_messages = [
    {"role": "system", "content": "You are a system that summarizes and analyzes medical records."},
    {"role": "user", "content": "From the following medical reports, please provide a summary and analysis of the patient" + 
                    f"\n{medical_reports_content}"}
]

In [23]:
summary_analysis_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=summary_analysis_messages
)
summary_and_analysis = summary_analysis_response["choices"][0]["message"]["content"]
print(summary_and_analysis)

Summary: 

The patient, whose name, gender, and date of birth are undisclosed, has a medical history of age-related degeneration and previous joint injuries. The patient presented with complaints of joint pain and limited mobility, particularly in the affected joints. Physical examination revealed mild swelling and tenderness in the affected joints, as well as restricted range of motion. X-rays and blood tests were ordered to rule out other potential conditions. Suspected age-related degeneration and previous joint injuries were identified as the potential causes of the joint pain and limited mobility. Pain medication was prescribed and physical therapy was recommended for pain management and improved mobility. After completing two weeks of physical therapy, the patient reported moderate improvement in joint pain and mobility. Physical examination showed reduced swelling and tenderness, along with increased range of motion compared to the previous visit. The patient was advised to cont